In [54]:
import cv2
import numpy as np
import torch
from PIL import Image
from torchvision import transforms
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [56]:
img = cv2.imread("../../image/lola/speleothem-lola-big.tif")
x_axis = (img.shape[0]//256)*256
y_axis = (img.shape[1]//256)*256

tiles = []

for x in range(0, x_axis, 256):
    for y in range(0, y_axis, 256):
        tiles.append(img[x:x+256, y:y+256])

In [60]:
data = []

for tile in tiles:
    input_image = Image.fromarray(tile)
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    input_tensor = preprocess(input_image)
    input_batch = input_tensor.unsqueeze(0)
    data.append(input_batch)

In [92]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=True)
newmodel = torch.nn.Sequential(*(list(model.children())[:-3]))
list(newmodel.children())[-1]

Using cache found in /home/diler/.cache/torch/hub/pytorch_vision_v0.10.0


Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample): Sequential(
      (0): Conv2d(128, 256, kernel_size=(1, 1), stride=(2, 2), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): BasicBlock(
    (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(256, eps=1

In [96]:
knn_data = []

for input_batch in data:
    with torch.no_grad():
        output = model(input_batch)
    knn_data.append(output.numpy().reshape(1000,))
knn_data = np.array(knn_data)


In [101]:
kmeans = KMeans(n_clusters=20, random_state=5)
kmeans.fit(knn_data)

/home/diler/miniconda3/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=20, random_state=5)